# Этап 1. Получение ссылок на книги автора

In [102]:
import re
import pandas as pd
import requests
import json
from multiprocessing import Pool
from bs4 import BeautifulSoup
from selenium import webdriver
import html
from time import sleep
from operator import setitem

from collections import defaultdict

In [47]:
authors_id = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}
author_url_format = "https://www.bookvoed.ru/author/books?id={}"
book_url_format = "https://www.bookvoed.ru/book?id={}"

In [103]:
# books_author = {
#   "Дарья Донцова":  [],
#   "Джеймс Роллинс": [],
#   "Макс Фрай":      [],
#   "Эрин Хантер":    [],
#   "Дмитрий Емец":   []
# }
books_author = defaultdict(list)

In [49]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

def get_books(driver, uid, t_sleep=1):
    url = author_url_format.format(uid)
    driver.get(url)
    sleep(t_sleep)
    
    last_height = driver.get_window_size()
    while(True):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(t_sleep)
        new_height = driver.execute_script("return document.body.scrollHeight")        
        try:
            play = driver.find_element_by_class_name('ty')
            play.click()
            sleep(t_sleep)
        except:
            pass
        if new_height == last_height:
            break
        else:
            last_height = new_height
    
    result = []
    items = driver.find_elements_by_class_name("gf")
    for item in items:
        book_id = item.get_attribute("data-book")
        result.append(book_id)
        
    return result


In [50]:
with webdriver.Firefox() as driver:
    for author, author_id in authors_id.items():
        books_author[author] = get_books(driver, author_id)

In [51]:
print(len(books_author["Дарья Донцова"]))
# 6424720
'6424720' in books_author["Дарья Донцова"]

954


False

# Этап 2. Получение информации о книгах

In [85]:
ages = {
    "ov nM": "0+",
    "pv nM": "6+",
    "qv nM": "12+",
    "rv nM": "16+",
    "sv nM": "18+"
}

pull_attributes_int = [
   "Год",
   "Страниц",
   "Код",
   "Тираж"
]

pull_attributes_str = [
    "Серия",
    "Издательство",
    "Переплёт",
    "ISBN",
    "Размеры",
    "Формат",
    "В базе",
    "Автор",
    "Тематика",
    "Переводчик",
    "Производитель"
]

In [69]:
from functools import wraps
from multiprocessing import Pool, Lock, Value
from time import sleep

mutex = Lock()
n_processed = Value('i', 0)
author_id = Value('i', 0)

def logger(func):
    @wraps(func)
    def wrapper(*args, **kwds):
        res = func(*args, **kwds)
        with mutex:
            global n_processed
            n_processed.value += 1
            if n_processed.value % 10 == 0:
                print(f"\r{n_processed.value} books are processed...", end='', flush=True)
        return res
    return wrapper

def hard_logger(func):
    @wraps(func)
    def wrapper(uid):
        with mutex:
            global n_processed
            n_processed.value += 1
#             if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} books are processed..." + str(uid), end='', flush=True)
        res = func(uid)
        
        return res
    return wrapper

In [94]:
@logger
def get_book_info(uid):
    
    book_url = book_url_format.format(uid)
    book_html = requests.get(book_url).text
    soup = BeautifulSoup(book_html, 'html.parser')
    age = None
    for k, v in ages.items():
        if soup.find('div', class_=k) not is None:
            age = v
            break
    desc = soup.find('div', class_='lw')
    price = soup.find('div', class_='Hu Wu')
    
    rating = soup.find('div', class_='He xe ')
    liked = rating.find('a', class_='Ke Me ').text.strip()
    bookmarks = rating.find('a', class_='Ke Le ff')
    bookmarks = bookmarks.text.strip() if bookmarks not is None else ''
    unliked = rating.find('a', class_='Ke Oe ').text.strip()
    
    res = {
        "ID": uid,
        "Название": str(soup.find('h1', itemprop='name').contents[0]),
        "Автор": str(soup.find('span', class_='Av').text),
        "Обложка": soup.find('img', class_='tf')['src'],
        "Возраст": age,
        "Описание": re.sub('\s+', ' ', desc.text) if desc != None else '',
        "Рейтинг": float(rating.find('div', class_='af')['style'][7:-1]),
        "Понравилось": int(liked) if liked else 0,
        "В закладки": int(bookmarks) if bookmarks else 0,
        "Не понравилось": int(unliked) if unliked else 0,
        "Цена": float((re.sub('\s+', '', str(price.contents[-1]))).strip()[:-4]) if price != None else '',
    }
    
    table = soup.find('table', class_='tw')
    rows = table.find_all('tr')
    data = [list(map(lambda x: re.sub("\s+", ' ', x.text.strip()), row.find_all('td')))
        for row in rows]
    data = dict(map(lambda x: (x[0][:-1], x[1]), data))
    *map(lambda x: setitem(data, x, 0) if x not in data else x, pull_attributes_int),
    *map(lambda x: setitem(data, x, '') if x not in data else x, pull_attributes_str),
    res.update(data)
    return res

In [95]:
result = []

for _, books in books_author.items():
    with Pool(processes=10) as pool: 
        result += pool.map(get_book_info, books)
        n_processed.value = 0
        author_id.value += 1

290 books are processed...

In [96]:
result[:2]


[{'ID': '9340995',
  'Название': 'Вечный двигатель маразма ',
  'Автор': 'Донцова Дарья Аркадьевна',
  'Обложка': '/files/1836/59/23/24/8.jpeg',
  'Возраст': '16+',
  'Описание': ' Самая опасная профессия – это писатель! А вы как думали? Вот врывается в офис к мужу Виолы Таракановой некая тетка потрепанной наружности, швыряет едва ли не в лицо брошюру и орет благим матом. Понятно, что у Виолы, как у каждого мало-мальски известного литератора, есть свой личный шизофреник. Но посетительница оказалась ее одноклассницей Любкой Гаскониной! И она утверждает, что Вилка еще в школьные годы завидовала ей и поэтому накропала книжонку, в которой были смачно расписаны всякие семейные тайны и пороки самой Любки и ее мамули – балерины на пенсии. Виола заявила, что подобную дрянь она никогда бы не стала писать - не тот жанр и формат. Тем не менее, муж Вилки, Степан Дмитриев, посчитал своим долгом отстоять честь жены, и шаг за шагом начал погружаться в семейные разборки семейства Гаскониных. Ох, если 

In [97]:
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)

In [98]:
df.head(4)

,ID,ISBN,Автор,Авторы,В базе,В закладки,Возраст,Год,Издательство,Код,...,Понравилось,Производитель,Размеры,Рейтинг,Серия,Страниц,Тематика,Тираж,Формат,Цена
831,2576314,978-5-699-64381-3,Донцова Дарья Аркадьевна,NaN,Э.ДиД.(м).Муха в самолете.Главбух и полцарства,0,16+,2013,Эксмо,960533,...,5,,"10,30 см x 16,40 см",61.54,Двойной артефакт-детектив,608,Отечественные,0,165.00mm x 105.00mm x 35.00mm,
1712,2576586,978-5-699-62817-9,Емец Дмитрий Александрович,NaN,Э.ДетВП.Король хитрости,1,6+,2013,Эксмо,962393,...,6,,"21,50 см x 14,50 см x 2,30 см",75.00,,336,Для младшего школьного возраста,4100,215.00mm x 145.00mm x 22.00mm,
1052,2579796,978-5-699-64916-7,Роллинс Джеймс,NaN,Э.Покет.Последний оракул,10,16+,2013,Эксмо,965104,...,24,,"11,50 см x 18,00 см x 2,20 см",88.89,Pocket book,544,Зарубежная,3000,180.00mm x 115.00mm x 24.00mm,
830,2579802,978-5-699-63186-5,Донцова Дарья Аркадьевна,NaN,Э.ИД.Кнопка управления мужем,1,16+,2013,Эксмо,965460,...,22,,"12,00 см x 19,50 см x 2,00 см",81.48,Иронический детектив,352,Отечественные,250000,196.00mm x 120.00mm x 21.00mm,


In [99]:
    with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)